In [331]:
import pandas as pd
import bs4 as bs
import re
import requests
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
from time import sleep

## Original Search Results

In [47]:
# import list of search strings
search_list = pd.read_csv('./sample_list.csv', skiprows=2)

In [93]:
# change spaces to %20 as prescribed in the new search url and store on list
query_list = search_list['Query'].str.replace(' ', '%20')
query_list = list(set(query_list))

In [94]:
# define query url
query_url = 'https://forum.cirruspilots.org/search?q='

In [138]:
# define site login function
def site_login():
    '''
    This is a helper function that logins to cirruspilots.org
    '''
    driver.get('https://passport.cirruspilots.org/')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtUsername').send_keys('jloretizo')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtPassword').send_keys('cth4h#@EoY*a')
    driver.find_element_by_id('login_btn').click()

In [325]:
# define the driver and execute login 
driver = webdriver.Safari()
site_login()

In [326]:
driver.get('https://www.cirruspilots.org/Forums')

In [337]:
# get the results set
results_set = {}
for i in tqdm(query_list):
    driver.get(query_url + i, )
    sleep(2)
    soup = bs.BeautifulSoup(driver.page_source, 'html.parser')
    links_list = []
    for j in soup.find_all(class_='fps-result'):
        href = j.find(class_='search-link')['href']
        links_list.append(href)
    results_set[i] = links_list
    sleep(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [368]:
def get_post_ids(row):
    ref_nums = []
    for i in range(len(row)):
        try:
            num = re.findall(r'\/[0-9]+\/[0-9]|\/[\d]+$', str(row[i]))[0]
        except:
            num = None
        ref_nums.append(num)
    return ref_nums

In [370]:
results_df = pd.DataFrame.from_dict(results_set, orient='index')
# results_df = results_df.reset_index()
results_df['ref_nums'] = results_df.apply(
    lambda row: get_post_ids(row), axis=1)

In [371]:
# converting links to dictionary
links_dict = {}
for i in results_df.index:
    links_dict[i] = results_df.loc[i]['ref_nums']

In [388]:
# stacking and resetting new dataframe
link_ids = pd.DataFrame.from_dict(
    links_dict, orient='index').stack().reset_index()

In [389]:
# column rename and search string renames
link_ids['level_0'] = link_ids['level_0'].apply(
    lambda x: x.replace('%20', ' '))
link_ids = link_ids.rename(columns={'level_0': 'search string', 0: 'id'})

In [393]:
# saving data to csv
link_ids[['search string', 'id']].to_csv('./search_results.csv', index=False)

## New Search Results

In [4]:
# open the search html template
with open('search.html', 'r') as open_file:
    soup = bs.BeautifulSoup(open_file, 'html.parser')

In [37]:
# find all content item classes
for i in soup.find_all(class_='content-item'):
    # change the value of title inside the a tag
    i.find(class_='abbreviated-post').find('h4').find('a')['title'] = 'New Title'
    # change the href for link
    i.find(class_='abbreviated-post').find('h4').find('a')['href'] = 'New Title'
    # change text value
    i.find(class_='abbreviated-post').find('h4').find('a').string.replace_with("No longer bold")
    

In [38]:
# save this new html as test panel
with open("output1.html", "w") as file:
    file.write(str(soup))

In [41]:
soup.find_all(class_='content-item')[0].find('h4')

<h4 class="post-name"><a class="internal-link view-post" href="https://www.cirruspilots.org/copa/safety_programs/f/349/t/159914.aspx" title="New Title">No longer bold</a></h4>